In [ ]:
# Instalações e downloads necessários

!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq

!wget -q https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

!pip install -q findspark

import findspark
findspark.init()

^C
^C
^C

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


Exception: Unable to find py4j in /content/spark-3.5.5-bin-hadoop3/python, your SPARK_HOME may not be configured correctly

In [ ]:
# Importações

from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
# Criando objeto SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Lendo o dataframe

df = spark.read.parquet("yellow_tripdata_2024-01.parquet")

In [ ]:
# Verificando o dataframe

df.describe().show()

In [ ]:
# Verificando a existência de dados inconsistentes

df.createOrReplaceTempView("yellow_tripdata")
resultado = spark.sql("SELECT * FROM yellow_tripdata WHERE passenger_count <= 0")
resultado.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-01-01 00:30:40|  2024-01-01 00:58:40|              0|          3.0|         1|                 N|         246|         231|           2|       25.4|  3.5|    0.5|       0.

In [ ]:
# Limpando dados inconsistentes

df = df.filter(df['passenger_count'] > 0)
df.select('passenger_count').show()

+---------------+
|passenger_count|
+---------------+
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              2|
|              1|
|              1|
|              2|
|              2|
|              2|
|              2|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
+---------------+
only showing top 20 rows



In [ ]:
# Verificando a existência de dados inconsistentes

df.createOrReplaceTempView("yellow_tripdata")
resultado = spark.sql("SELECT * FROM yellow_tripdata WHERE trip_distance <= 0")
resultado.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:52:09|  2024-01-01 00:52:28|              1|          0.0|         1|                 N|         237|         237|           2|        3.0|  1.0|    0.5|       0.

In [ ]:
# Limpando dados inconsistentes

df = df.filter(df['trip_distance'] > 0)
df.select('trip_distance').show()

+-------------+
|trip_distance|
+-------------+
|         1.72|
|          1.8|
|          4.7|
|          1.4|
|          0.8|
|          4.7|
|        10.82|
|         5.44|
|         0.04|
|         0.75|
|          1.2|
|          8.2|
|          0.4|
|          0.8|
|          5.0|
|          1.5|
|          1.5|
|         2.57|
|         0.66|
|          1.7|
+-------------+
only showing top 20 rows



In [ ]:
# Verificando a existência de dados inconsistentes

df.createOrReplaceTempView("yellow_tripdata")
resultado = spark.sql("SELECT * FROM yellow_tripdata WHERE fare_amount < 0")
resultado.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+---------

In [ ]:
# Limpando dados inconsistentes

df = df.filter(df['fare_amount'] >= 0)
df.select('fare_amount').show()

+-----------+
|fare_amount|
+-----------+
|       17.7|
|       10.0|
|       23.3|
|       10.0|
|        7.9|
|       29.6|
|       45.7|
|       31.0|
|        3.0|
|        7.9|
|       14.9|
|       59.0|
|        5.8|
|        6.5|
|       21.2|
|       12.8|
|        9.3|
|       17.7|
|        6.5|
|       29.6|
+-----------+
only showing top 20 rows



In [ ]:
from functools import reduce
from pyspark.sql.functions import col

# Verificando a existência de dados nulos

conds = [col(c).isNull() for c in df.columns]
any_null = reduce(lambda a, b: a | b, conds)

df.filter(any_null).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:18:24|  2024-01-01 00:30:39|              1|         2.16|         1|                 N|         249|         232|           4|      -13.5| -1.0|   -0.5|       0.

In [ ]:
# Limpando dados inconsistentes

df = df.filter(df['extra'] >= 0)
df.select('extra').show()

+-----+
|extra|
+-----+
|  1.0|
|  3.5|
|  3.5|
|  3.5|
|  3.5|
|  3.5|
|  6.0|
|  1.0|
|  1.0|
|  1.0|
|  3.5|
|  3.5|
|  3.5|
|  3.5|
| 2.75|
|  3.5|
|  1.0|
|  1.0|
|  1.0|
|  3.5|
+-----+
only showing top 20 rows



In [ ]:
# Limpando dados inconsistentes

df = df.filter(df['mta_tax'] >= 0)
df.select('mta_tax').show()

+-------+
|mta_tax|
+-------+
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
|    0.5|
+-------+
only showing top 20 rows



In [ ]:
# Limpando dados inconsistentes

df = df.filter((df['tip_amount'] >= 0) & (df['congestion_surcharge'] >= 0) & (df['Airport_fee'] >= 0))
df.select('tip_amount', 'congestion_surcharge', 'Airport_fee').show()

+----------+--------------------+-----------+
|tip_amount|congestion_surcharge|Airport_fee|
+----------+--------------------+-----------+
|       0.0|                 2.5|        0.0|
|      3.75|                 2.5|        0.0|
|       3.0|                 2.5|        0.0|
|       2.0|                 2.5|        0.0|
|       3.2|                 2.5|        0.0|
|       6.9|                 2.5|        0.0|
|      10.0|                 0.0|       1.75|
|       0.0|                 2.5|        0.0|
|       0.0|                 2.5|        0.0|
|       0.0|                 2.5|        0.0|
|      3.95|                 2.5|        0.0|
|     14.15|                 2.5|        0.0|
|      1.25|                 2.5|        0.0|
|       0.0|                 2.5|        0.0|
|       0.0|                 0.0|       1.75|
|      4.45|                 2.5|        0.0|
|       3.0|                 2.5|        0.0|
|      10.0|                 2.5|        0.0|
|      2.88|                 2.5| 

In [ ]:
# Limpando dados inconsistentes. RatecodeID = 99 é um erro segundo pesquisas na internet

df = df.filter(df['RatecodeID'] != 99)
df.select('RatecodeID').show()

+----------+
|RatecodeID|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 20 rows



In [ ]:
# Limpando dados inconsistentes

df = df.filter(df['improvement_surcharge'] >= 0)
df.select('improvement_surcharge').show()

+---------------------+
|improvement_surcharge|
+---------------------+
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
|                  1.0|
+---------------------+
only showing top 20 rows



In [ ]:
# Limpando dados desnecessários

df = df.drop("VendorID",
"passenger_count",
"store_and_fwd_flag",
"payment_type")


In [ ]:
# Analisando o df

df.describe().show()

In [ ]:
# Importa as bibliotecas necessárias para ML

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors

# Cria um Vetor de Features
# Fare_amount, tip_amount, tolls_amount atrapalham nosso modelo já que interferem diretamente no amount final

assembler = VectorAssembler(
    inputCols=[
        "trip_distance",
        "RatecodeID",
        "PULocationID",
        "DOLocationID",
        "extra",
        "mta_tax",
        "improvement_surcharge",
        "congestion_surcharge",
        "Airport_fee"
    ],
    outputCol="features"
)
df_prepared = assembler.transform(df)

# Dividir em treino e teste

train_data, test_data = df_prepared.randomSplit([0.7, 0.3], seed=42)

# Definir o modelo de regressão

lr = LinearRegression(
    featuresCol="features",
    labelCol="total_amount"  # Agora é uma coluna numérica!
)

# Treinar o modelo

model = lr.fit(train_data)

# Fazer previsões
predictions = model.transform(test_data)

# Avaliar métricas
print(f"RMSE no treino: {model.summary.rootMeanSquaredError:.2f}")
print(f"R² no treino: {model.summary.r2:.2f}")

# Mostrar previsões
predictions.select("total_amount", "prediction").show(10)

RMSE no treino: 13.11
R² no treino: 0.64
+------------+------------------+
|total_amount|        prediction|
+------------+------------------+
|         9.4|16.947972038477154|
|        28.6|20.799088699023017|
|        13.5|20.202002590721293|
|        21.6| 21.95087902269389|
|       11.25|24.284993610715013|
|       16.32| 20.50994623898049|
|         7.6|18.194003128728752|
|       25.55| 23.68813272452433|
|       20.52|  23.4476654310198|
|       92.69| 89.90351849030733|
+------------+------------------+
only showing top 10 rows



In [ ]:
# Testando outros modelos

from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Dividir em treino e teste
train, test = df_prepared.randomSplit([0.7, 0.3], seed=42)

# Definir o evaluator de métricas
evaluator_rmse = RegressionEvaluator(
    labelCol="total_amount", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(
    labelCol="total_amount", predictionCol="prediction", metricName="r2")

# Decision Tree Regressor
dt = DecisionTreeRegressor(featuresCol="features", labelCol="total_amount")
model_dt = dt.fit(train)
pred_dt = model_dt.transform(test)
rmse_dt = evaluator_rmse.evaluate(pred_dt)
r2_dt   = evaluator_r2.evaluate(pred_dt)

print(f"Decision Tree — RMSE: {rmse_dt:.2f}, R²: {r2_dt:.2f}")

# Random Forest Regressor
rf = RandomForestRegressor(featuresCol="features", labelCol="total_amount", numTrees=50)
model_rf = rf.fit(train)
pred_rf = model_rf.transform(test)
rmse_rf = evaluator_rmse.evaluate(pred_rf)
r2_rf   = evaluator_r2.evaluate(pred_rf)

print(f"Random Forest  — RMSE: {rmse_rf:.2f}, R²: {r2_rf:.2f}")

# Gradient-Boosted Trees Regressor
gbt = GBTRegressor(featuresCol="features", labelCol="total_amount", maxIter=50)
model_gbt = gbt.fit(train)
pred_gbt = model_gbt.transform(test)
rmse_gbt = evaluator_rmse.evaluate(pred_gbt)
r2_gbt   = evaluator_r2.evaluate(pred_gbt)

print(f"Gradient Boost — RMSE: {rmse_gbt:.2f}, R²: {r2_gbt:.2f}")

# Comparar resultados
results = [
    ("Decision Tree", rmse_dt, r2_dt),
    ("Random Forest", rmse_rf, r2_rf),
    ("Gradient Boost", rmse_gbt, r2_gbt),
]
for name, rmse, r2 in results:
    print(f"{name:15s} | RMSE = {rmse:.2f} | R² = {r2:.2f}")


Decision Tree — RMSE: 6.71, R²: 0.91
Random Forest  — RMSE: 7.60, R²: 0.88
Gradient Boost — RMSE: 6.38, R²: 0.92
Decision Tree   | RMSE = 6.71 | R² = 0.91
Random Forest   | RMSE = 7.60 | R² = 0.88
Gradient Boost  | RMSE = 6.38 | R² = 0.92


Nosso melhor modelo foi o Gradient Boost com R^2 = 0.92